In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")


In [6]:
import ell
ell.init(store='./log', autocommit=True)

In [7]:
@ell.simple( model="gpt-4o-mini", temperature=0.7 )
def name_meaning(name: str) -> str:
    """
    You are a chatbot that tells the meaning of person's name in a funny way.
    """
    return f"The person who approached you is named {name}."




In [8]:
if __name__ == '__main__':
    print( name_meaning("Mordecai") )

Ah, Mordecai! The name that sounds like it should belong to a wise old wizard or perhaps the ruler of an ancient kingdom where everyone wears capes and has epic mustaches. The name actually means "warrior" or "servant of Marduk," which is fitting because with a name like that, you either end up conquering kingdoms or serving snacks at your local game night. So, whether you're swinging a sword or just swinging by to grab a slice of pizza, Mordecai has got some serious vibes going on!


In [9]:
@ell.simple( model="gpt-4o-mini", temperature=0.7 )
def name_meaning(name: str, age: int) -> str:
    """
    You are a chatbot that tells the meaning of person's name in a funny way.
    Adapt your response to the person's age.
    """
    return f"The person who approached you is named {name}. Their age is {age}."

In [10]:
if __name__ == '__main__':
    print(name_meaning("Mordecai", 20))
    print(name_meaning("Mordecai", 5))
    

Ah, Mordecai! The name that sounds like it could belong to a wise old owl or a hipster barista who serves artisanal coffee with a sprinkle of wisdom! At 20, you might not have the beard of a sage just yet, but your name suggests you’re destined to have deep conversations about life while sipping on something fancy. Just remember, if you ever start giving advice to people, make sure it’s not just about which video game to play next!
Oh, Mordecai! What a big name for such a little superstar! It sounds like a wizard's name, doesn’t it? It’s like you’re destined to cast spells and make candy rain from the sky! So, watch out—one day you might just turn your broccoli into chocolate! Keep being awesome, little wizard! ✨🍭


## Defining tools for agents

In [11]:
import math
from pydantic import Field

In [12]:
@ell.tool()
def sin(angle: float = Field(description="Angle in degrees.")) -> str:
    """Returns a value of sin(angle) with up 10 decimal places. For example sin(30) = 0.5."""
    value = math.sin(math.radians(angle))
    return f"{value:.10f}"

In [13]:
from typing import List


In [14]:
@ell.complex(model="gpt-4o-mini", tools=[ sin ], temperature=0.5)
def math_assistant(message_history: List[ell.Message]) -> List[ell.Message]:
    return [
        ell.system("You are a helpful assistant that helps with math."),
    ] + message_history

In [15]:
def converse(initial_prompt: str):
    conversation = [ ell.user(initial_prompt) ]
    response: ell.Message = math_assistant(conversation)
    return response.text

In [16]:
if __name__ == '__main__':
    print(converse("What is sin(30)?"))

ToolCall(sin(angle=30.0), tool_call_id='call_S9seJh3IgNzOoGfhIH395Oi2')


In [24]:
def converse(initial_prompt: str):
    conversation = [ ell.user(initial_prompt) ]
    response: ell.Message = math_assistant(conversation)
    max_iterations = 5
    # ToolCall 체크를 response.tool_calls만으로 하면 됩니다
    while max_iterations > 0 and response.tool_calls:
        tool_results = response.call_tools_and_collect_as_message()
        conversation = conversation + [response, tool_results]
        response = math_assistant(conversation)
        max_iterations -= 1

    if max_iterations <= 0:
        raise Exception("Too many iterations, probably stuck in a loop.")
  
    return response.text

In [20]:
if __name__ == '__main__':
    print(converse("What is sin(30)?"))

The value of sin(30) is 0.5.


In [23]:
if __name__ == '__main__':
    print("First question: ", converse("What is sin(35) and sin(51)?"))
    print("Second question: ", converse("What is sin(sin(67))?"))
    

First question:  The values are:
- \( \sin(35^\circ) \approx 0.5735764364 \)
- \( \sin(51^\circ) \approx 0.7771459615 \)
Second question:  The value of \( \sin(\sin(67)) \) is approximately \( 0.0160651494 \).


In [28]:
if __name__ == '__main__':
    print("First question: ", converse("What is sine of sine of sine of 10 degrees?"))
    print("Second question: ", converse("What is sine of sine of sine of sine of sine of sine of 10 degrees?"))
    

First question:  The sine of sine of sine of 10 degrees is approximately \( 0.0000528962 \).
Second question:  The sine of sine of sine of sine of sine of sine of 10 degrees is approximately \( 0.0000009232 \).


In [29]:
def _sin(angle: float) -> float:
    return math.sin(math.radians(angle))

def _tan(angle: float) -> float:
    return math.tan(math.radians(angle))

def _number_comparator(a: float, b: float) -> str:
    if a > b:
        return f"{a} is larger than {b}, {a} > {b}"
    elif a < b:
        return f"{a} is smaller than {b}, {a} < {b}"
    else:
        return f"{a} is equal to {b}, {a} = {b}"

In [31]:
@ell.tool()
def number_comparator(a: float = Field(description="First number 'a' to compare."),
                      b: float = Field(description="Second number 'b' to compare.")) -> str:
    """Returns 'larger' if a > b, 'smaller' if a < b and 'equal' if a = b."""
    return _number_comparator(a, b)

@ell.tool()
def tangent(angle: float = Field(description="Angle in degrees.")) -> str:
    """Returns a value of tan(angle), which is tangent, with up 10 decimal places. For example tan(45) = 1."""
    value = _tan(angle)
    return f"{value:.10f}"

@ell.tool()
def sin(angle: float = Field(description="Angle in degrees.")) -> str:
    """Returns a value of sin(angle), which is sine, with up 10 decimal places. For example sin(30) = 0.5."""
    value = _sin(angle)
    return f"{value:.10f}"

In [32]:
@ell.complex(model="gpt-4o-mini", tools=[ sin, tangent, number_comparator ], temperature=0.5)
def math_assistant(message_history: List[ell.Message]) -> List[ell.Message]:
    return [
        ell.system("You are a helpful assistant that helps with math."),
    ] + message_history